## Helper Functions

### CBAuth

In [1]:
import http.client
import hmac
import hashlib
import json
import time
from urllib.parse import urlencode
from typing import Union, Dict


class CBAuth:
    """
    Singleton class for Coinbase authentication.
    """

    _instance = None  # Class attribute to hold the singleton instance

    def __new__(cls):
        """
        Override the __new__ method to control the object creation process.
        :return: A single instance of CBAuth
        """
        if cls._instance is None:
            print("Authenticating with Coinbase")
            cls._instance = super(CBAuth, cls).__new__(cls)
            cls._instance.init()
        return cls._instance

    def init(self):
        """
        Initialize the CBAuth instance with API credentials.
        """
        self.key = None
        self.secret = None

    def set_credentials(self, api_key, api_secret):
        """
        Update the API credentials used for authentication.
        :param api_key: The API Key for Coinbase API
        :param api_secret: The API Secret for Coinbase API
        """
        self.key = api_key
        self.secret = api_secret

    def __call__(self, method: str, path: str, body: Union[Dict, str] = '', params: Dict[str, str] = None) -> Dict:
        """
        Prepare and send an authenticated request to the Coinbase API.

        :param method: HTTP method (e.g., 'GET', 'POST')
        :param path: API endpoint path
        :param body: Request payload
        :param params: URL parameters
        :return: Response from the Coinbase API as a dictionary
        """
        path = self.add_query_params(path, params)
        body_encoded = self.prepare_body(body)
        headers = self.create_headers(method, path, body)
        return self.send_request(method, path, body_encoded, headers)

    def add_query_params(self, path, params):
        if params:
            query_params = urlencode(params)
            path = f'{path}?{query_params}'
        return path

    def prepare_body(self, body):
        return json.dumps(body).encode('utf-8') if body else b''

    def create_headers(self, method, path, body):
        timestamp = str(int(time.time()))
        message = timestamp + method.upper() + \
            path.split('?')[0] + (json.dumps(body) if body else '')
        signature = hmac.new(self.secret.encode(
            'utf-8'), message.encode('utf-8'), digestmod=hashlib.sha256).hexdigest()

        return {
            "Content-Type": "application/json",
            "CB-ACCESS-KEY": self.key,
            "CB-ACCESS-SIGN": signature,
            "CB-ACCESS-TIMESTAMP": timestamp
        }

    def send_request(self, method, path, body_encoded, headers):
        conn = http.client.HTTPSConnection("api.coinbase.com")
        try:
            conn.request(method, path, body_encoded, headers)
            res = conn.getresponse()
            data = res.read()

            if res.status == 401:
                print("Error: Unauthorized. Please check your API key and secret.")
                return None

            response_data = json.loads(data.decode("utf-8"))
            if 'error_details' in response_data and response_data['error_details'] == 'missing required scopes':
                print(
                    "Error: Missing Required Scopes. Please update your API Keys to include more permissions.")
                return None

            return response_data
        except json.JSONDecodeError:
            print("Error: Unable to decode JSON response. Raw response data:", data)
            return None
        finally:
            conn.close()


### API Key Management

In [2]:
import pandas as pd

api_csv = pd.read_csv('coinbase-api-key.csv')

API_KEY = api_csv['API_KEY'][0]
API_SECRET = api_csv['API_SECRET'][0]

In [3]:
def set_api_credentials(api_key=None, api_secret=None):
    global API_KEY
    global API_SECRET

    # Option 1: Use provided arguments
    if api_key and api_secret:
        API_KEY = api_key
        API_SECRET = api_secret
        
    # Update the CBAuth singleton instance with the new credentials
    CBAuth().set_credentials(API_KEY, API_SECRET)


In [4]:
set_api_credentials(API_KEY, API_SECRET)

Authenticating with Coinbase


### Coinbase Client

In [5]:
from enum import Enum
from datetime import datetime
import uuid
import json

# Initialize the single instance of CBAuth
cb_auth = CBAuth()


class Side(Enum):
    BUY = 1
    SELL = 0


class Method(Enum):
    POST = "POST"
    GET = "GET"


def generate_client_order_id():
    return str(uuid.uuid4())


def listAccounts(limit=49, cursor=None):
    """
    Get a list of authenticated accounts for the current user.

    This function uses the GET method to retrieve a list of authenticated accounts from the Coinbase Advanced Trade API.

    :param limit: A pagination limit with default of 49 and maximum of 250. If has_next is true, additional orders are available to be fetched with pagination and the cursor value in the response can be passed as cursor parameter in the subsequent request.
    :param cursor: Cursor used for pagination. When provided, the response returns responses after this cursor.
    :return: A dictionary containing the response from the server. A successful response will return a 200 status code. An unexpected error will return a default error response.
    """
    return cb_auth(
        Method.GET.value,
        "/api/v3/brokerage/accounts",
        {"limit": limit, "cursor": cursor},
    )


def getAccount(account_uuid):
    """
    Get a list of information about an account, given an account UUID.

    This function uses the GET method to retrieve information about an account from the Coinbase Advanced Trade API.

    :param account_uuid: The account's UUID. Use listAccounts() to find account UUIDs.
    :return: A dictionary containing the response from the server. A successful response will return a 200 status code. An unexpected error will return a default error response.
    """
    return cb_auth(Method.GET.value, f"/api/v3/brokerage/accounts/{account_uuid}")


def createOrder(client_order_id, product_id, side, order_type, order_configuration):
    """
    Create an order with the given parameters.

    :param client_order_id: A unique ID generated by the client for this order.
    :param product_id: The ID of the product to order.
    :param side: The side of the order (e.g., 'buy' or 'sell').
    :param order_type: The type of order (e.g., 'limit_limit_gtc').
    :param order_configuration: A dictionary containing order details such as price, size, and post_only.
    :return: A dictionary containing the response from the server.
    """
    payload = {
        "client_order_id": client_order_id,
        "product_id": product_id,
        "side": side,
        "order_configuration": {order_type: order_configuration},
    }
    # print("Payload being sent to server:", payload)  # For debugging
    return cb_auth(Method.POST.value, "/api/v3/brokerage/orders", payload)


def cancelOrders(order_ids):
    """
    Initiate cancel requests for one or more orders.

    This function uses the POST method to initiate cancel requests for one or more orders on the Coinbase Advanced Trade API.

    :param order_ids: A list of order IDs for which cancel requests should be initiated.
    :return: A dictionary containing the response from the server. A successful response will return a 200 status code. An unexpected error will return a default error response.
    """
    body = json.dumps({"order_ids": order_ids})
    return cb_auth(Method.POST.value, "/api/v3/brokerage/orders/batch_cancel", body)


def listOrders(**kwargs):
    """
    Retrieve a list of historical orders.

    This function uses the GET method to retrieve a list of historical orders from the Coinbase Advanced Trade API.
    The orders are returned in a batch format.

    :param kwargs: Optional parameters that can be passed to the API. These can include:
        'product_id': Optional string of the product ID. Defaults to null, or fetch for all products.
        'order_status': A list of order statuses.
        'limit': A pagination limit with no default set.
        'start_date': Start date to fetch orders from, inclusive.
        'end_date': An optional end date for the query window, exclusive.
        'user_native_currency': (Deprecated) String of the users native currency. Default is `USD`.
        'order_type': Type of orders to return. Default is to return all order types.
        'order_side': Only orders matching this side are returned. Default is to return all sides.
        'cursor': Cursor used for pagination.
        'product_type': Only orders matching this product type are returned. Default is to return all product types.
        'order_placement_source': Only orders matching this placement source are returned. Default is to return RETAIL_ADVANCED placement source.
        'contract_expiry_type': Only orders matching this contract expiry type are returned. Filter is only applied if ProductType is set to FUTURE in the request.
    :return: A dictionary containing the response from the server. This will include details about each order, such as the order ID, product ID, side, type, and status.
    """
    return cb_auth(
        Method.GET.value, "/api/v3/brokerage/orders/historical/batch", params=kwargs
    )


def listFills(**kwargs):
    """
    Retrieve a list of fills filtered by optional query parameters.

    This function uses the GET method to retrieve a list of fills from the Coinbase Advanced Trade API.
    The fills are returned in a batch format.

    :param kwargs: Optional parameters that can be passed to the API. These can include:
        'order_id': Optional string of the order ID.
        'product_id': Optional string of the product ID.
        'start_sequence_timestamp': Start date. Only fills with a trade time at or after this start date are returned.
        'end_sequence_timestamp': End date. Only fills with a trade time before this start date are returned.
        'limit': Maximum number of fills to return in response. Defaults to 100.
        'cursor': Cursor used for pagination. When provided, the response returns responses after this cursor.
    :return: A dictionary containing the response from the server. This will include details about each fill, such as the fill ID, product ID, side, type, and status.
    """
    return cb_auth(
        Method.GET.value, "/api/v3/brokerage/orders/historical/fills", params=kwargs
    )


def getOrder(order_id):
    """
    Retrieve a single order by order ID.

    This function uses the GET method to retrieve a single order from the Coinbase Advanced Trade API.

    :param order_id: The ID of the order to retrieve.
    :return: A dictionary containing the response from the server. This will include details about the order, such as the order ID, product ID, side, type, and status.
    """
    return cb_auth(Method.GET.value, f"/api/v3/brokerage/orders/historical/{order_id}")


def listProducts(**kwargs):
    """
    Get a list of the available currency pairs for trading.

    This function uses the GET method to retrieve a list of products from the Coinbase Advanced Trade API.

    :param limit: An optional integer describing how many products to return. Default is None.
    :param offset: An optional integer describing the number of products to offset before returning. Default is None.
    :param product_type: An optional string describing the type of products to return. Default is None.
    :param product_ids: An optional list of strings describing the product IDs to return. Default is None.
    :param contract_expiry_type: An optional string describing the contract expiry type. Default is 'UNKNOWN_CONTRACT_EXPIRY_TYPE'.
    :return: A dictionary containing the response from the server. This will include details about each product, such as the product ID, product type, and contract expiry type.
    """
    return cb_auth(Method.GET.value, "/api/v3/brokerage/products", params=kwargs)


def getProduct(product_id):
    """
    Get information on a single product by product ID.

    This function uses the GET method to retrieve information about a single product from the Coinbase Advanced Trade API.

    :param product_id: The ID of the product to retrieve information for.
    :return: A dictionary containing the response from the server. This will include details about the product, such as the product ID, product type, and contract expiry type.
    """
    response = cb_auth(Method.GET.value, f"/api/v3/brokerage/products/{product_id}")

    # Check if there's an error in the response
    if "error" in response and response["error"] == "PERMISSION_DENIED":
        print(f"Error: {response['message']}. Details: {response['error_details']}")
        return None

    return response


def getProductCandles(product_id, start, end, granularity):
    """
    Get rates for a single product by product ID, grouped in buckets.

    This function uses the GET method to retrieve rates for a single product from the Coinbase Advanced Trade API.

    :param product_id: The trading pair.
    :param start: Timestamp for starting range of aggregations, in UNIX time.
    :param end: Timestamp for ending range of aggregations, in UNIX time.
    :param granularity: The time slice value for each candle.
    :return: A dictionary containing the response from the server. This will include details about each candle, such as the open, high, low, close, and volume.
    """
    params = {"start": start, "end": end, "granularity": granularity}
    return cb_auth(
        Method.GET.value,
        f"/api/v3/brokerage/products/{product_id}/candles",
        params=params,
    )


def getMarketTrades(product_id, limit):
    """
    Get snapshot information, by product ID, about the last trades (ticks), best bid/ask, and 24h volume.

    This function uses the GET method to retrieve snapshot information about the last trades from the Coinbase Advanced Trade API.

    :param product_id: The trading pair, i.e., 'BTC-USD'.
    :param limit: Number of trades to return.
    :return: A dictionary containing the response from the server. This will include details about the last trades, such as the best bid/ask, and 24h volume.
    """
    return cb_auth(
        Method.GET.value,
        f"/api/v3/brokerage/products/{product_id}/ticker",
        {"limit": limit},
    )


def getTransactionsSummary(
    start_date,
    end_date,
    user_native_currency="USD",
    product_type="SPOT",
    contract_expiry_type="UNKNOWN_CONTRACT_EXPIRY_TYPE",
):
    """
    Get a summary of transactions with fee tiers, total volume, and fees.

    This function uses the GET method to retrieve a summary of transactions from the Coinbase Advanced Trade API.

    :param start_date: The start date of the transactions to retrieve, in datetime format.
    :param end_date: The end date of the transactions to retrieve, in datetime format.
    :param user_native_currency: The user's native currency. Default is 'USD'.
    :param product_type: The type of product. Default is 'SPOT'.
    :param contract_expiry_type: Only orders matching this contract expiry type are returned. Only filters response if ProductType is set to 'FUTURE'. Default is 'UNKNOWN_CONTRACT_EXPIRY_TYPE'.
    :return: A dictionary containing the response from the server. This will include details about each transaction, such as fee tiers, total volume, and fees.
    """
    params = {
        "start_date": start_date.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "end_date": end_date.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "user_native_currency": user_native_currency,
        "product_type": product_type,
        "contract_expiry_type": contract_expiry_type,
    }
    return cb_auth(Method.GET.value, "/api/v3/brokerage/transaction_summary", params)


### Spot Price

In [6]:
def get_spot_price(product_id):
    """
    Fetches the current spot price of a specified product.

    Args:
        product_id (str): The ID of the product (e.g., "BTC-USD").

    Returns:
        float: The spot price as a float, or None if an error occurs.
    """
    try:
        response = client.getProduct(product_id)
        # print("Response:", response)  # Log the entire response for debugging
        quote_increment = Decimal(response['quote_increment'])

        # Check whether the 'price' field exists in the response and return it as a float
        if 'price' in response:
            price = Decimal(response['price'])
            # Round the price to quote_increment number of digits
            rounded_price = price.quantize(quote_increment)
            return rounded_price
        else:
            # Print a specific error message if the 'price' field is missing
            print(f"'price' field missing in response for {product_id}")
            return None

    except Exception as e:
        print(f"Error fetching spot price for {product_id}: {e}")
        return None

## Strategy Functions

### Limit Buy

In [7]:
from decimal import Decimal, ROUND_HALF_UP

# Initialize the single instance of CBAuth
cb_auth = CBAuth()

BUY_PRICE_MULTIPLIER = 0.99

def fiat_limit_buy(product_id, fiat_amount, price_multiplier=BUY_PRICE_MULTIPLIER):
    """
    Places a limit buy order.

    Args:
        product_id (str): The ID of the product to buy (e.g., "BTC-USD").
        fiat_amount (float): The amount in USD or other fiat to spend on buying (ie. $200).
        price_multiplier (float, optional): Multiplier to apply to the current spot price to get the limit price. Defaults to BUY_PRICE_MULTIPLIER.

    Returns:
        dict: The response of the order details.
    """
    # Coinbase maker fee rate
    maker_fee_rate = Decimal("0.006")

    # Fetch product details to get the quote_increment and base_increment
    product_details = getProduct(product_id)
    print(product_details)
    # Check if 'quote_increment' is in product_details
    if "quote_increment" not in product_details:
        print(f"Error: 'quote_increment' not found in product details for {product_id}")
        return None

    quote_increment = Decimal(product_details["quote_increment"])
    base_increment = Decimal(product_details["base_increment"])

    # Fetch the current spot price for the product
    spot_price = get_spot_price(product_id)

    # Calculate the limit price
    limit_price = Decimal(spot_price) * Decimal(price_multiplier)

    # Round the limit price to the appropriate number of decimal places
    limit_price = limit_price.quantize(quote_increment)

    # Adjust the fiat_amount for the maker fee
    effective_fiat_amount = Decimal(fiat_amount) * (1 - maker_fee_rate)

    # Calculate the equivalent amount in the base currency (e.g., BTC) for the given USD amount
    base_size = effective_fiat_amount / limit_price

    # Round base_size to the nearest allowed increment
    base_size = (base_size / base_increment).quantize(
        Decimal("1"), rounding=ROUND_HALF_UP
    ) * base_increment

    # Create order configuration
    order_configuration = {
        "limit_price": str(limit_price),
        "base_size": str(base_size),
        "post_only": True,
    }

    # Send the order
    order_details = createOrder(
        client_order_id=generate_client_order_id(),
        product_id=product_id,
        side=Side.BUY.name,
        order_type="limit_limit_gtc",
        order_configuration=order_configuration,
    )
    print(order_details)

    # Check if 'success' is in order_details
    if "success" not in order_details:
        print(f"Error: 'success' not found in order details for {product_id}")
        return None

    # Print a human-readable message
    if order_details["success"]:
        base_size = Decimal(
            order_details["order_configuration"]["limit_limit_gtc"]["base_size"]
        )
        limit_price = Decimal(
            order_details["order_configuration"]["limit_limit_gtc"]["limit_price"]
        )
        total_amount = base_size * limit_price
        print(
            f"Successfully placed a limit buy order for {base_size} {product_id} (${total_amount:.2f}) at a price of {limit_price} USD."
        )
    else:
        print(
            f"Failed to place a limit buy order. Reason: {order_details['failure_reason']}"
        )

    print("Coinbase response:", order_details)

    return order_details

### Limit Sell

In [8]:
from decimal import Decimal, ROUND_HALF_UP

# Initialize the single instance of CBAuth
cb_auth = CBAuth()

SELL_PRICE_MULTIPLIER = 1.01

def fiat_limit_sell(product_id, fiat_amount, price_multiplier=SELL_PRICE_MULTIPLIER):
    """
    Places a limit sell order.

    Args:
        product_id (str): The ID of the product to sell (e.g., "BTC-USD").
        fiat_amount (float): The amount in USD or other fiat to receive from selling (ie. $200).
        price_multiplier (float, optional): Multiplier to apply to the current spot price to get the limit price. Defaults to SELL_PRICE_MULTIPLIER.

    Returns:
        dict: The response of the order details.
    """
    # Coinbase maker fee rate
    maker_fee_rate = Decimal("0.006")

    # Fetch product details to get the quote_increment and base_increment
    product_details = getProduct(product_id)
    quote_increment = Decimal(product_details["quote_increment"])
    base_increment = Decimal(product_details["base_increment"])

    # Fetch the current spot price for the product
    spot_price = get_spot_price(product_id)

    # Calculate the limit price
    limit_price = Decimal(spot_price) * Decimal(price_multiplier)

    # Round the limit price to the appropriate number of decimal places
    limit_price = limit_price.quantize(quote_increment)

    # Adjust the fiat_amount for the maker fee
    effective_fiat_amount = Decimal(fiat_amount) / (1 - maker_fee_rate)

    # Calculate the equivalent amount in the base currency (e.g., BTC) for the given USD amount
    base_size = effective_fiat_amount / limit_price

    # Round base_size to the nearest allowed increment
    base_size = (base_size / base_increment).quantize(
        Decimal("1"), rounding=ROUND_HALF_UP
    ) * base_increment

    # Create order configuration
    order_configuration = {
        "limit_price": str(limit_price),
        "base_size": str(base_size),
        "post_only": True,
    }

    # Send the order
    order_details = createOrder(
        client_order_id=generate_client_order_id(),
        product_id=product_id,
        side=Side.SELL.name,
        order_type="limit_limit_gtc",
        order_configuration=order_configuration,
    )

    # Print a human-readable message
    if order_details["success"]:
        base_size = Decimal(
            order_details["order_configuration"]["limit_limit_gtc"]["base_size"]
        )
        limit_price = Decimal(
            order_details["order_configuration"]["limit_limit_gtc"]["limit_price"]
        )
        total_amount = base_size * limit_price
        print(
            f"Successfully placed a limit sell order for {base_size} {product_id} (${total_amount:.2f}) at a price of {limit_price} USD."
        )
    else:
        print(
            f"Failed to place a limit sell order. Reason: {order_details['failure_reason']}"
        )

    print("Coinbase response:", order_details)

    return order_details


### Fear-Grade Index

In [9]:
import requests

# Default schedule for the trade_based_on_fgi_simple function
schedule = [
    {"threshold": 10, "factor": 1.5, "action": "buy"},
    {"threshold": 20, "factor": 1.3, "action": "buy"},
    {"threshold": 30, "factor": 1.1, "action": "buy"},
    {"threshold": 70, "factor": 0.9, "action": "sell"},
    {"threshold": 80, "factor": 0.7, "action": "sell"},
    {"threshold": 90, "factor": 0.5, "action": "sell"},
]

def get_fear_and_greed_index():
    """
    Fetches the latest Fear and Greed Index (FGI) values from the API.

    Returns:
        tuple: A tuple containing the FGI value and its classification.
    """
    response = requests.get("https://api.alternative.me/fng/?limit=1")
    data = response.json()["data"][0]
    return int(data["value"]), data["value_classification"]
    
def trade_based_on_fgi(product_id, fiat_amount, schedule=schedule):
    """
    Executes a trade based on the Fear and Greed Index (FGI) using a professional strategy.

    Args:
        product_id (str): The ID of the product to trade.
        fiat_amount (float): The amount of fiat currency to trade.
        schedule (list, optional): The trading schedule. Defaults to PRO_SCHEDULE.

    Returns:
        dict: The response from the trade execution.
    """

    fgi, classification = get_fear_and_greed_index()

    # Use the provided schedule 
    schedule = schedule

    for condition in schedule:
        if fgi <= condition["threshold"]:
            fiat_amount *= condition["factor"]
            if condition["action"] == "buy":
                response = fiat_limit_buy(product_id, fiat_amount)
            else:
                response = fiat_limit_sell(product_id, fiat_amount)
            return {
                **response,
                "Fear and Greed Index": fgi,
                "classification": classification,
            }

In [10]:
get_fear_and_greed_index()

(72, 'Greed')